In [ ]:
!pip install taichi -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00


Taichi example on github

In [ ]:
# python/taichi/examples/simulation/fractal.py

import taichi as ti

ti.init(arch=ti.gpu)

n = 320
pixels = ti.field(dtype=float, shape=(n * 2, n))


@ti.func
def complex_sqr(z):
    return ti.Vector([z[0]**2 - z[1]**2, z[1] * z[0] * 2])


@ti.kernel
def paint(t: float):
    for i, j in pixels:  # Parallelized over all pixels
        c = ti.Vector([-0.8, ti.cos(t) * 0.2])
        z = ti.Vector([i / n - 1, j / n - 0.5]) * 2
        iterations = 0
        while z.norm() < 20 and iterations < 50:
            z = complex_sqr(z) + c
            iterations += 1
        pixels[i, j] = 1 - iterations * 0.02


gui = ti.GUI("Julia Set", res=(n * 2, n))

for i in range(1000000):
    paint(i * 0.03)
    gui.set_image(pixels)
    gui.show()

[Taichi] version 1.7.0, llvm 15.0.4, commit 2fd24490, linux, python 3.10.12
[Taichi] Starting on arch=cuda


RuntimeError: [x11.cpp:create_window@154] Taichi fails to create a window. This is probably due to the lack of an X11 GUI environment. Consider using the `ti.GUI(show_gui=False)` option, see https://docs.taichi-lang.org/docs/gui_system

Taichi fractal image example supported on colab

In [ ]:
!mkdir fractal_images

In [ ]:
import taichi as ti
import os
import imageio
from PIL import Image

ti.init(arch=ti.gpu)
n = 320
pixels = ti.field(dtype=float, shape=(n * 2, n))

@ti.func
def complex_sqr(z):
    return ti.Vector([z[0]**2 - z[1]**2, z[1] * z[0] * 2])

@ti.kernel
def paint(t: float):
    for i, j in pixels:
        c = ti.Vector([-0.8, ti.cos(t) * 0.2])
        z = ti.Vector([i / n - 1, j / n - 0.5]) * 2
        iterations = 0
        while z.norm() < 20 and iterations < 50:
            z = complex_sqr(z) + c
            iterations += 1
        pixels[i, j] = 1 - iterations * 0.02


for i in range(1000):
    paint(i * 0.03)
    img_data = (pixels.to_numpy() * 255).astype(np.uint8)
    img = Image.fromarray(img_data)
    img_path = os.path.join('fractal_images', f'fractal_{i:04d}.png')
    img.save(img_path)


with imageio.get_writer('fractal.mp4', mode='I') as writer:
    for i in range(1000):
        img_path = os.path.join('fractal_images', f'fractal_{i:04d}.png')
        img = imageio.imread(img_path)
        writer.append_data(img)

[Taichi] Starting on arch=cuda


<ipython-input-10-decded0fb741>:37: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(img_path)
